# TEI Conference 2012 #

Le programme actuel a pour but de récupérer les abstracts de chaque conférence de la TEI Conference 2012, de les normaliser puis de les analyser via des méthodes de machine learning.

## Etape 1 : le téléchargement. ##

L'étape actuelle consiste à télécharger depuis le site web les données des abstracts puis de créer :
- Un fichier csv listant le titre, les auteurs, l'université d'origine et le nom de l'abstract dans le cache
- Un cache dans lequel les textes sont stockés en format texte brut.

Nous n'avons pas besoin d'une phase de normalisation (Etape 2 dans les autres années) car nous n'avons pas de format XML utilisé. La page HTML est directement parsée et le texte brut en est extrait sans étapes intermédiaires.

### A. Les packages ###

Il faut installer les packages ci-dessous en ne le lançant qu'une fois. 

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import json
import os
from tqdm import tqdm
from lxml import etree
#Créer un certificat SSL pour sécuriser l'accès au site internet.
import ssl 

ssl._create_default_https_context = ssl._create_unverified_context 

### B. Téléchargement et stockage ###

En lançant la case ci-dessous, vous téléchargez le contenu dans la page web, puis :
- Le contenu est trié et un document csv est créé, contenant le titre de tous les abstracts, le nom des auteurs, l'université de rattachement si elle existe et le nom de l'abstract dans le cache.
- Tous les abstracts sont stockés dans un cache au format .txt

Le titre des abstracts dans le cache a ce format : paper-2012-1, où seul le dernier chiffre change entre les abstracts.

In [4]:
entiers = ['1', '2', '3', '4', '5', '6', '7', '8', '9']#liste permettant d'écarter plus tard dans le texte les chiffres gênants et inutiles

content = urlopen('https://tei-c.org/Vault/MembersMeetings/2012/teiconference/program/papers/#mads')
HTML = content.read()
# La suite de code en .replace consiste à corriger manuellement des erreurs et des irrégularités dans le code HTML
# Je prends le HTML, le transforme de bytes en str, et j'y amène des corrections. Autrement, c'est impossible à traiter logiquement
HTML = HTML.decode('UTF-8')
HTML = HTML.replace('<p><a name="grad"></a><br />', '')
HTML = HTML.replace('<strong>Conversation, Translation', '<p><a name="grad"></a><strong>Conversation, Translation')
HTML = HTML.replace('<p><a name="encfin"></a></p>', '')
HTML = HTML.replace('<p><strong>Encoding Financial', '<p><a name="encfin"></a><strong>Encoding Financial')
HTML = HTML.replace('<p><a name="xpoint"></a><strong>Rebooting TEI XPointers</strong><br />', '<p><a name="xpoint"></a><strong>Rebooting TEI XPointers</strong><br /></p>')
HTML = HTML.replace('<strong><span style="text-decoration: underline">Hugh Cayless', '<p><strong><span style="text-decoration: underline">Hugh Cayless')
HTML = HTML.replace('<strong><a name="ets"></a>', '<a name="ets"></a><strong>')
HTML = HTML.replace('<strong><a name="librarysupport"></a>', '<a name="librarysupport"></a><strong>')
HTML = HTML.replace('<p><a name="mads">', '<hr/><p><a name="mads">')

#Je crée mon cache et un de ses sous-dossiers
if not os.path.exists("./cache2012"):
    os.makedirs("./cache2012")
if not os.path.exists("./cache2012/cacheTXT"):
    os.makedirs("./cache2012/cacheTXT")
Path = './cache2012/cacheTXT/'


masterList = []

#Je vais trier l'information, en obtenant des strings. J'utilise la balise <hr/> qui sépare chaque abstract
soup = BeautifulSoup(HTML, 'html.parser')
liste_des_abstracts = {}
i = 1
j = 0
for hr in tqdm(soup.find_all("hr")):
    texte = []
    for item in hr.find_next_siblings():
        liste_nom_direct = []
        liste_nom_indirect = []
        liste_institution = []
        if item.name == "hr":
            i = 1
            break
        if item.name == "p":
            if i == 1: #la seule logique est celle de la suite des <p> : le premier est le titre, le 2e les auteur.es, le 3e est l'université s'il y en a une. Je crée donc un index suivi
                if item.string == None:
                    if len(item.contents) == 1: #item.contents est une liste où chaque élément est un couple de balise ouvrante/fermante, item.string est une str. Seul le couple de balise contenant le texte m'intéresse.
                        titre = item.string
                    elif len(item.contents) == 2:
                        titre_tags = item.contents[1]
                        titre = titre_tags.string
                    elif len(item.contents) == 3:
                        titre_tags = item.contents[1]
                        titre = titre_tags.string
                else:
                    i = i - 1 #permet de sauter une ligne si la première ligne ne contient pas le nom de l'auteur
            if i == 2:
                for elem in item.contents: #Le problème consiste ici à trouver les balises comportant le nom de l'auteur, car elles sont entourées de balises parasites qui ne m'intéressent pas
                    if elem.string != None: #permet d'écarter les balises ne contenant pas de texte
                        if len(elem.string) > 2:
                            if elem.string[-2] not in entiers: #permet d'écarter les chiffres renvoyant à une note de base de page
                                nom_direct = elem.string
                                liste_nom_direct.append(nom_direct)#permet de relever les différents noms dans le cas d'auteurs multiples
                    else:
                        for souselem in elem.contents: #permet de trouver les balises contenues dans des balises
                            if souselem.string not in entiers:
                                nom_indirect = souselem.string
                                liste_nom_indirect.append(nom_indirect)
                noms_indirect = ' '.join(liste_nom_indirect)
                noms_direct = ' '.join(liste_nom_direct)
                noms = noms_direct + ' ' + noms_indirect  #Je réunis ici les noms trouvés par toutes les manières dans une seule str
                noms = noms.replace('2,1', '') #un élément perturbateur que je n'arrivais à dégager est retiré
            if i == 3:
                for elem in item.contents: #je vais ensuite trier par le contenu pour sélectionner les éléments intéressants concernant la provenance universitaire
                    if "Universi" in elem or "Ecole" in elem or "ILIT" in elem or "Institut" in elem or "College" in elem: #permet de savoir si l'élément étudié contient les termes usuels désignant une université dans les langues européennes et nord-américaines
                        institution = elem
                        liste_institution.append(institution)
                    else:
                        if "Texas A&M University" in noms: #je recherche et corrige une erreur dans l'encodage que je n'arrivais pas à corriger : l'université était encodé comme un nom
                            noms = noms.replace("Texas A&M University", "")
                            institution = "Texas A&M University"
                            liste_institution.append(institution)
                        else:
                            institutions = ' ' #certains auteurs ne sont pas rattachés à une université. Je mets un espace vide pour qu'il y ait quand même une case vide dans le tableau.
                institutions = ' '.join(liste_institution)
            else:
                if item.string != None: #permet de relever toutes les balises contenant directement du texte. Les autres balises "déchets" contiennent les noms d'auteurs, les titres, les adresses mails, et quelques éléments éparses dont la complexité pour les attraper ne vaut pas leur valeur et leur importance.
                    paragraphe = item.string
                    texte.append(paragraphe)
            i += 1
    liste_finale = [] #j'instancie la liste qui regroupe les éléments d'autorité
    titre_dans_cache = "paper-2012-{}".format(j)
    j += 1
    abstract = " ".join(texte)
    with open('./cache2012/cacheTXT/%s.txt' % titre_dans_cache, 'w', encoding='UTF-8') as docTXT: #j'écris dans un cache les abstracts avec leur titre
        docTXT.write(titre)
        docTXT.write('\n')
        docTXT.write(abstract)
    liste_finale.append(titre)
    liste_finale.append(noms)
    liste_finale.append(institutions)
    liste_finale.append(titre_dans_cache) #permet de relier aisément l'abstract et son nom dans le cache
    masterList.append(liste_finale) #liste regroupant les informations d'un abstract
    
    
#je stocke la liste des infos de chaque abstract dans un csv
with open('./cache2012/TEI2012.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    print("Ecriture du csv terminée")
    for abstract in tqdm(masterList):
        writer.writerow(abstract)
         

100%|██████████| 24/24 [00:00<00:00, 4506.98it/s]

Ecriture du csv terminée
